# 04 Get Metadata

> Get medata from files.

In [ ]:
#|default_exp core.04_metadata

In [ ]:
#|hide
import nbdev; nbdev.nbdev_export()

In [ ]:
#|hide
from nbdev.showdoc import show_doc

In [ ]:
#|export
import pubcrawler as proj
from pubcrawler import const, log, utils, tools
import adu_proj.utils as adutils

In [ ]:
#|export
import json
import PyPDF2
from openai import OpenAI
from dotenv import load_dotenv
import os
from tqdm import tqdm
import pandas as pd
import functools
import time
import signal
from functools import wraps

In [ ]:
#|export
load_dotenv()
API_KEY = os.getenv("OPENAI_API_KEY")
client = OpenAI(
    api_key=API_KEY,
)

We will attempt to get metadata from each file. Currently this script assumes the file is a pdf although in the future its possible to add new methods here for extracting data from different file types. 

Passing entire documents to an LLM can be expensive so we instead only pass a subsection (first 5 pages). 

In [ ]:
#|export
with open(f'{const.pre_output_path}/data_files.json', 'r') as f:
    file_links = json.load(f)

In [ ]:
#|export
def extract_text_from_first_n_pages(filepath, n):
    text = ""
    with open(filepath, 'rb') as file:
        reader = PyPDF2.PdfReader(file)
        for i in range(min(n, len(reader.pages))):
            page = reader.pages[i]
            text += page.extract_text()
    return text

Lets now get metadata from this section of pdf via function calling:

In [ ]:
#|export
class TimeoutException(Exception):
    pass

def timeout_handler(signum, frame):
    raise TimeoutException

def retry_on_timeout(retries=3, timeout=30):
    def decorator(func):
        @wraps(func)
        def wrapper(*args, **kwargs):
            for _ in range(retries):
                try:
                    signal.signal(signal.SIGALRM, timeout_handler)
                    signal.alarm(timeout)
                    result = func(*args, **kwargs)
                    signal.alarm(0)  # Reset the alarm
                    return result
                except TimeoutException:
                    print(f"Function {func.__name__} timed out, retrying...")
                finally:
                    signal.alarm(0)  # Ensure the alarm is cleared
            raise TimeoutException(f"Function {func.__name__} failed after {retries} retries")
        return wrapper
    return decorator

In [ ]:
#|export
@retry_on_timeout(retries=2, timeout=30)
def get_pdf_metadata(report_text):
    functions = [{
            "name": "generate_citation",
            "description": "Generate a citation for report",
            "parameters": {
                "type": "object",
                "properties": {
                    "title": {
                        "type": "string",
                        "description": "Report title. If unknown leave empty",
                    },
                    "authors": {
                        "type": "array",
                        "description": "Array of the author's full names. If unknown leave empty",
                        "items": {
                                "description": "Author's full name",
                                "type": "string"
                            }
                    },
                    "organisation": {
                        "type": "array",
                        "description": "Array containing the names of the research organisations that produced the report. If unknown leave empty",
                        "items": {
                                "description": "Research organisation's name",
                                "type": "string"
                            }
                    },
                    "date": {
                        "type": "string",
                        "description": "Date the report was published on. Try to find the day, month and year. If unknown leave empty",
                    },
                    "keywords": {
                        "type": "array",
                        "description": "Array of keywords that indicate the content of the report. If unknown leave empty",
                        "items": {
                                "description": "Keyword title ie 'feminism'",
                                "type": "string"
                            }
                    },
                    "funders": {
                        "type": "array",
                        "description": "Array of organisations that provided funding or financial support. If unknown leave empty",
                        "items": {
                                "description": "Name of funding organisation",
                                "type": "string"
                            }
                    }
                },
                "required": ["title", "authors", "organisation", "date", "keywords", "funders"]
            },
        }]
    messages = []
    messages.append({"role": "system", "content": "Don't make assumptions about what values to plug into functions. Ask for clarification if a user request is ambiguous."})
    messages.append({"role": "user", "content": f"Generate a citation for the report based on the following excerpt: {report_text}"})
    chat_response = client.chat.completions.create(
        messages=messages, functions=functions, model='gpt-4'
    )
    assistant_response = json.loads(chat_response.choices[0].message.function_call.arguments)
    return assistant_response    

Now we can loop through all the files getting metadata:

In [ ]:
#|export
for key in tqdm(file_links):
    try:
        m = get_pdf_metadata(extract_text_from_first_n_pages(file_links[key]['file_path'], 5))
        file_links[key]['title'] = m['title']
        file_links[key]['authors'] = m['authors']
        file_links[key]['organisation'] = m['organisation']
        file_links[key]['date'] = m['date']
        file_links[key]['keywords'] = m['keywords']
        file_links[key]['funders'] = m['funders']
    except:
        print(f"failed on {key}")

 12%|██████▌                                               | 23/190 [04:36<41:02, 14.74s/it]

failed on https://www.homeless.org.uk/sites/default/files/site-attachments/Homeless%20Link%20and%20St%20Mungo%20briefing%20on%20Universal%20Credit%20-%20money%20management%20debate.pdf
failed on https://hummedia.manchester.ac.uk/institutes/cresc/research/WDTMG%20FINAL%20-01-3-2016.pdf


 17%|█████████▍                                            | 33/190 [05:58<31:13, 11.93s/it]

failed on https://orca.cf.ac.uk/18439/1/Quality%20%26%20Independence%20of%20British%20Journalism.pdf


 28%|███████████████                                       | 53/190 [09:06<25:49, 11.31s/it]

...

failed on http://www.autonomyinstitute.org/wp-content/uploads/2017/07/Work-and-gender-V6.pdf


 97%|███████████████████████████████████████████████████▌ | 185/190 [30:03<00:40,  8.10s/it]

failed on http://www.datascienceassn.org/sites/default/files/The%20Scored%20Society%20-%20Due%20Process%20for%20Automated%20Predictions.pdf


100%|█████████████████████████████████████████████████████| 190/190 [30:31<00:00,  9.64s/it]

failed on http://www.autonomyinstitute.org/wp-content/uploads/2017/07/Precarious-work-V8.pdf
failed on http://www.autonomyinstitute.org/wp-content/uploads/2017/07/Automation-V5.pdf


In [ ]:
#|export
pub_metadata = pd.DataFrame([file_links[key] for key in file_links]).dropna()

In [ ]:
#|export
pub_metadata.to_csv(f'{const.pre_output_path}/pub_metadata.csv', index=False)